<a href="https://colab.research.google.com/github/Shifoue/Segmentation/blob/main/Iris_Mask_Dataset_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 48.8 MB/s eta 0:00:00


In [2]:
!unzip Dataset_Faces.zip

Archive:  Dataset_Faces.zip
   creating: Dataset_Faces/
  inflating: Dataset_Faces/000001.jpg  
  inflating: Dataset_Faces/000002.jpg  
  inflating: Dataset_Faces/000003.jpg  
  inflating: Dataset_Faces/000004.jpg  
  inflating: Dataset_Faces/000005.jpg  
  inflating: Dataset_Faces/000006.jpg  
  inflating: Dataset_Faces/000007.jpg  
  inflating: Dataset_Faces/000008.jpg  
  inflating: Dataset_Faces/000009.jpg  
  inflating: Dataset_Faces/000010.jpg  
  inflating: Dataset_Faces/000011.jpg  
  inflating: Dataset_Faces/000012.jpg  
  inflating: Dataset_Faces/000013.jpg  
  inflating: Dataset_Faces/000014.jpg  
  inflating: Dataset_Faces/000015.jpg  
  inflating: Dataset_Faces/000016.jpg  
  inflating: Dataset_Faces/000017.jpg  
  inflating: Dataset_Faces/000018.jpg  
  inflating: Dataset_Faces/000019.jpg  
  inflating: Dataset_Faces/000020.jpg  
  inflating: Dataset_Faces/000021.jpg  
  inflating: Dataset_Faces/000022.jpg  
  inflating: Dataset_Faces/000023.jpg  
  inflating: Dataset_Fac

In [3]:
import numpy as np
import mediapipe as mp
import cv2 as cv
import os
import glob

from google.colab.patches import cv2_imshow

In [4]:
def Iris_Segmentation(image):
    img = image.copy()
    mp_face_mesh = mp.solutions.face_mesh

    LEFT_IRIS = [474,475, 476, 477]
    RIGHT_IRIS = [469, 470, 471, 472]

    with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as face_mesh:
        rgb_frame = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img_h, img_w = img.shape[:2]
        results = face_mesh.process(rgb_frame)
        mask = np.zeros((img_h, img_w), dtype=np.uint8)

        if results.multi_face_landmarks:
            
            mesh_points=np.array([np.multiply([p.x, p.y], [img_w, img_h]).astype(int) 
            for p in results.multi_face_landmarks[0].landmark])
            
            (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
            (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
            center_left = np.array([l_cx, l_cy], dtype=np.int32)
            center_right = np.array([r_cx, r_cy], dtype=np.int32)
            cv.circle(image, center_left, int(l_radius), (0,255,0), 2, cv.LINE_AA)
            cv.circle(image, center_right, int(r_radius), (0,255,0), 2, cv.LINE_AA)

            cv.circle(mask, center_left, int(l_radius), (255,255,255), -1, cv.LINE_AA)
            cv.circle(mask, center_right, int(r_radius), (255,255,255), -1, cv.LINE_AA)

    return mask, image

In [5]:
def load_images(path):
    image_list = []
    for filename in glob.glob(path):
        im=cv.imread(filename)
        #im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
        image_list.append(im)

    return image_list

In [6]:
images = load_images("/content/Dataset_Faces/*.jpg")

In [7]:
mask_path = '/content/Dataset_Faces_Mask' 
if not os.path.exists(mask_path):
    os.makedirs(mask_path)

segmentation_path = '/content/Dataset_Faces_Segmented' 
if not os.path.exists(segmentation_path):
    os.makedirs(segmentation_path)

In [8]:
c = 1

for image in images:
    segmentation = Iris_Segmentation(image)

    cv.imwrite(mask_path + "/"+ str(c)+ ".jpg", segmentation[0])
    cv.imwrite(segmentation_path + "/"+ str(c)+ ".jpg", segmentation[1])

    c += 1